In [2]:
pip install openmeteo_requests


   ---------------------------------------- 0/3 [flatbuffers]
   ------------- -------------------------- 1/3 [openmeteo-sdk]
   -------------------------- ------------- 2/3 [openmeteo_requests]
   ---------------------------------------- 3/3 [openmeteo_requests]

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install retry_requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests_cache


   ---------------------------------------- 0/3 [url-normalize]
   ------------- -------------------------- 1/3 [cattrs]
   ------------- -------------------------- 1/3 [cattrs]
   ------------- -------------------------- 1/3 [cattrs]
   ------------- -------------------------- 1/3 [cattrs]
   ------------- -------------------------- 1/3 [cattrs]
   -------------------------- ------------- 2/3 [requests_cache]
   -------------------------- ------------- 2/3 [requests_cache]
   -------------------------- ------------- 2/3 [requests_cache]
   ---------------------------------------- 3/3 [requests_cache]

Note: you may need to restart the kernel to use updated packages.


In [5]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

In [6]:

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -0.983,
	"longitude": -78.618,
	"start_date": "2023-04-24",
	"end_date": "2025-05-08",
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_speed_100m", "relative_humidity_2m", "weather_code", "rain", "pressure_msl", "apparent_temperature", "precipitation", "cloud_cover", "temperature_2m_spread", "soil_temperature_0_to_7cm"],
	"timezone": "Pacific/Auckland"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(2).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(6).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(7).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(8).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(9).ValuesAsNumpy()
hourly_temperature_2m_spread = hourly.Variables(10).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(11).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["weather_code"] = hourly_weather_code
hourly_data["rain"] = hourly_rain
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["temperature_2m_spread"] = hourly_temperature_2m_spread
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates -0.9490333795547485°N -78.59951782226562°E
Elevation 2769.0 m asl
Timezone b'Pacific/Auckland'b'GMT+12'
Timezone difference to GMT+0 43200 s
                           date  temperature_2m  wind_speed_10m  \
0     2023-04-23 12:00:00+00:00       10.926499        3.396233   
1     2023-04-23 13:00:00+00:00       11.776500        2.189795   
2     2023-04-23 14:00:00+00:00       12.376500        5.091168   
3     2023-04-23 15:00:00+00:00       12.876500        2.902413   
4     2023-04-23 16:00:00+00:00       13.026500        4.334974   
...                         ...             ...             ...   
17899 2025-05-08 07:00:00+00:00        9.576500        1.537921   
17900 2025-05-08 08:00:00+00:00        9.176499        3.438895   
17901 2025-05-08 09:00:00+00:00        8.626500        3.572898   
17902 2025-05-08 10:00:00+00:00        8.476500        2.800143   
17903 2025-05-08 11:00:00+00:00        7.726500        1.853213   

       wind_speed_100m  relative_humidity_

In [8]:
data = hourly_dataframe.to_csv('dataClimaProyecto.csv', index=False)

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import shutil

shutil.move("/content/dataClima.csv", "/content/drive/MyDrive/UTPL/HerramientasIA/dataClima.csv")

'/content/drive/MyDrive/UTPL/HerramientasIA/dataClima.csv'